In [7]:
import pandas as pd
import numpy as np
import geopy

In [2]:

filepath = r'bbb'

# load file
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '//' + "{}".format(fName), sep =',', index=False)
    return print("Saved {} to {}".format(fName,filepath))

# load GTFS files
trips = load('trips')
stops = load('stops')
stop_times = load('stop_times')
shapes = load('shapes')



### Goal:
    Write a pre-processing script that can ingest GTFS and that will "inject" the stop lat and lon to the shapes lat and lon.

### Approach:
    Write logic for a test case using a single trip and route. 
Will need to match up the shape_id from shapes.txt with shape_id from trips.txt. Will need to get stops from the stop_times.txt and the lat lon
from the stops.txt. 

Loop through the shapes.txt and find where the lat and lon of the stop is in between shape ponts and inject in there? Then will need to reorder
the sequence. -Apply will be better than loop?

Use the distance from stop to the shape point in order to determine where the stop lat lon should be injected into the shapes.txt.
* Mesaure in Meters
* Look for the shortest distance for the stop to all of the shape points. Once found, inject the stop coords- replace the orig stop point with the lat lon of stop. 
    - ###### working on this 5/16
    - Logic goes like this:
        * create a new col with the distance to the stop, find the smallest number (will need to test to see if this works in all cases), calculate the correct
        seq, then inject into the new shapes dataframe.
        * using the calc_distance and insert_row functions to accomplish this 



In [3]:
# get one trip to test 
onetrip = trips[(trips.trip_id == 778854)]
oneshape = shapes[(shapes.shape_id == 23890)]
stopTimeStops = stop_times['stop_id'][(stop_times.trip_id == 778854)].tolist()
neededStops = stops[stops['stop_id'].isin(stopTimeStops)]
# oneshape

In [60]:
'''
Use example stop id 1389 'BARRINGTON SB & TEXAS NS'on route 2993 and trip 778854

Use the distance.geodesic function from geopy

Inject the stop coords into the shape.txt 

'''

# use example stop id 1389 'BARRINGTON SB & TEXAS NS'

def calc_distance(x):
    stop_coords = (34.047224, -118.459131) # hard coding stop lat lon to get 'injecting' logic working
    shape_coords = (x.shape_pt_lat, x.shape_pt_lon)
    return geopy.distance.geodesic(stop_coords, shape_coords ).m



# create df with the shape lat, lon, dist to stop, and pt seq
out_df_badIndex = pd.DataFrame({
    'shape_pt_lat' : oneshape.shape_pt_lat,
    'shape_pt_lon' : oneshape.shape_pt_lon,
    'shape_pt_sequence' : oneshape.shape_pt_sequence,
    'dist_to_stp': oneshape.apply(calc_distance, axis = 1) # output of calc_distance function
})
# re-index here b/c can't access the original index??
# is actually kind of nice b/c will be able to use iloc later on to get row, not index
# also, indexing dosen't matter as the shape_pt_sequence is the main col to keep in order
out_df = out_df_badIndex.reset_index(drop=True)
# grab to the index id of the df for the min value in the dist_to_stop column
row_to_update = out_df['dist_to_stp'].idxmin()
# grab the index id for the stop before the min vlue this is for pt_seq order
minus_one_row = row_to_upate - 1
# this is the point seq for the new shape point
new_pt_seq = out_df.iloc[[minus_one_row]]['shape_pt_sequence'].values[0] + 1


In [59]:

# Function (I found and modified) to insert row in the dataframe 'gracefully'
def Insert_row_(row_number, df, row_value): 
    df1 = df[0:row_number] 
    df2 = df[row_number:] 
    df1.loc[row_number]=row_value 
    df_result = pd.concat([df1, df2]) 
    df_result.index = [*range(df_result.shape[0])] 
    return df_result 

# row to update
row_number = row_to_update
# the pt seq to update
new_shape_pt_seq = new_pt_seq
# what will be 'injected'- stop lat lon here
row_value = [34.047224, -118.459131, new_shape_pt_seq, 0] 
  
if row_number > out_df.index.max()+1: 
    print("Invalid row_number") 
else: 
    # here, create a df and run the insert_row function, b/c that function does not override the existing row, need to drop
    # the original shape_point. This can be changed later, but if will work for now. Think of it as swapping
    # out the nearest shape point in the sequence with the lat lon of the stop. 
    df3 = pd.DataFrame(Insert_row_(row_number, out_df, row_value)).drop_duplicates(subset='shape_pt_sequence',keep='first',inplace=False)
    
    
df3[(df3['dist_to_stp'] < 200)]

    
# below is the final result, index 97 is now the lat lon of the stop lat lon- not worried about the warning 

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,shape_pt_lat,shape_pt_lon,shape_pt_sequence,dist_to_stp
92,34.048610,-118.460289,93.0,187.264718
93,34.048379,-118.460090,94.0,155.738350
94,34.048070,-118.459810,95.0,112.857040
95,34.047899,-118.459649,96.0,88.845780
96,34.047559,-118.459349,97.0,42.260819
97,34.047224,-118.459131,98.0,0.000000
99,34.046139,-118.458140,99.0,151.186541


In [ ]:
''' will I need a for loop in these scripts? or use apply?'''
# for index,row in out_df.iterrows():
#     print(row['shape_pt_lon'],row['shape_pt_lat'],row['dist_to_stp'])
    
#         print(out_df['dist_to_stp'])